In [2]:
# import thư viện
import os
import cv2
import numpy as np
from tensorflow.keras.models import Model, load_model
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
# Tính chỉ số IoU (Intersection over Union) và áp dụng để đánh giá mô hình.
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = label(y_true_in > 0.5)
    y_pred = label(y_pred_in > 0.5)
    
    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    #  Chuẩn bị dữ liệu để tính toán union (tổng hợp) giữa các đối tượng
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Tính union
    union = area_true + area_pred - intersection

    # Lọc bỏ nền
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Tính toán chỉ số iou
    iou = intersection / union
    # tính toán các tp,fp,fn dựa trên ngưỡng và iou
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # đối tượng đúng
        false_positives = np.sum(matches, axis=0) == 0  # đối tượng bị khuyết thiếu
        false_negatives = np.sum(matches, axis=1) == 0  # đối tượng phụ
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    #Tính toán các giá trị precision (độ chính xác) tương ứng với từng ngưỡng (threshold) IoU (Intersection over Union) thông qua vòng lặp for:
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)


# Tính trung bình các iou trên từng mẫu 
def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.array(np.mean(metric), dtype=np.float32)

# Đánh giá mô hình  trả về giá trị số thực
def my_iou_metric(label, pred):
    metric_value = tf.py_function(iou_metric_batch, [label, pred], tf.float32)
    return metric_value

# Đọc dữ liệu cho tập FISH_DATASET

In [5]:
path = "D:\\study\\machine_learning\\classification_fish\\data\\data_raw\\Fish_Dataset"

In [6]:
data = [] # mảng chứa dữ liệu
folders = os.listdir(path)
for folder in folders:
    folder_path = os.path.join(path, folder, folder)
    for i in range(1, 1001):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        data.append([img, folder])

In [7]:
images = []
names = []

for image, name in data:
    images.append(image)
    names.append(name)

In [8]:
# chuấn hóa label thành dạng số
le = LabelEncoder()
names = le.fit_transform(names)

# in ra label và tên tương ứng
for i in range(9):
    print(i, le.classes_[i])

0 Black Sea Sprat
1 Gilt Head Bream
2 Hourse Mackerel
3 Red Mullet
4 Red Sea Bream
5 Sea Bass
6 Shrimp
7 Striped Red Mullet
8 Trout


In [9]:
masks = [] # mảng chứa dữ liệu
folders = os.listdir(path)
for folder in folders:
    folder_gt = folder + " GT"
    folder_path = os.path.join(path, folder, folder_gt)
    for i in range(1, 1001):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (128, 128))
        masks.append(img)

In [10]:
images = np.array(images)
masks = np.array(masks)
masks = masks.reshape(9000, 128, 128, 1)
masks = masks.astype(bool)

# Load model unet1 để tạo ra các ảnh mask cho dữ liệu

In [12]:
model = load_model('D:\\study\\machine_learning\\classification_fish\\model\\segmeantation\\unet_1.hdf5', 
                   custom_objects={'my_iou_metric': my_iou_metric})

In [13]:
preds_ = model.predict(images, verbose=1)

282/282 [==============================] - 135s 480ms/step


In [14]:
preds_ = (preds_ > 0.5).astype(np.uint8)

In [15]:
output_folder = 'D:\\study\\machine_learning\\classification_fish\\data\\data_fix\\Fish_Dataset_GT'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [16]:
# Xác định các thư mục
folder_names = ['Black Sea Sprat', 'Gilt Head Bream', 'Hourse Mackerel', 'Red Mullet',
               'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']

# Tạo ra các thư mục nếu nó chưa tồn tại
for folder_name in folder_names:
    folder_path = os.path.join(output_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)

In [17]:
preds_ = np.squeeze(preds_)

In [ ]:
# Lặp qua các hình ảnh
for i in range(len(preds_)):
    # Lấy tên thư mục và chỉ mục ảnh
    folder_index = i // 1000
    folder_name = folder_names[folder_index]
    image_index = i % 1000

  # Định dạng chỉ mục ảnh dưới dạng chuỗi 5 chữ số
    image_index_str = str(image_index + 1).zfill(5)

    # Tạo tên ảnh
    filename = f'{image_index_str}.png'

    # Chuyển đổi về dạng nhị phân
    img_binary = (preds_[i] > 0.5).astype(np.uint8) * 255

    # Tạo đối tượng từ mảng nhị phân
    pred_image = Image.fromarray(img_binary, mode='L')

    # Lưu các ảnh
    folder_path = os.path.join(output_folder, folder_name)
    output_path = os.path.join(folder_path, filename)
    pred_image.save(output_path)

In [ ]:
path_img = "D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_raw\\Fish_Dataset"
path_gt = 'D:\\study\\machine_learning\\classification_fish\\data\\data_fix\\Fish_Dataset_GT'
output_folder = "D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_fix\\Fish_Dataset_Segment"

In [ ]:
# Define the folder names
folder_names = ['Black Sea Sprat', 'Gilt Head Bream', 'Hourse Mackerel', 'Red Mullet',
               'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']

# Create the folders if they don't exist
for folder_name in folder_names:
    folder_path = os.path.join(output_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)

In [ ]:
folders = os.listdir(path_img)
for folder in folders:
    folder_path = os.path.join(path_img,folder, folder)
    folder_gt = os.path.join(path_gt, folder)
    print(folder_path)
    print(folder_gt)
    for i in range(1, 1001):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")
        mask_path = os.path.join(folder_gt, f"{str(i).zfill(5)}.png")
        # đọc ảnh
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        mask = cv2.imread(mask_path)
        mask = cv2.resize(mask, (128, 128))

        # Chuyển ảnh mask sang ảnh binary
        gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)

        obj = np.zeros_like(img)

        # Tìm contours trên ảnh binary
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # Lặp qua từng contour
        for contour in contours:
            # Tìm bounding box của contour
            x, y, w, h = cv2.boundingRect(contour)

            # Kiểm tra kích thước bounding box
            if w > 5 and h > 5:
                # Kiểm tra màu của pixel tương ứng trên ảnh ground truth
                if mask[y:y + h, x:x + w].max() == 255:
                    # Lấy phần ảnh tương ứng từ ảnh gốc
                    obj[y:y + h, x:x + w] = cv2.bitwise_and(img[y:y + h, x:x + w], mask[y:y + h, x:x + w])
                    
        cv2.imwrite(f"D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_fix\\Fish_Dataset_Segment\\{folder}\\{str(i).zfill(5)}.png", obj)

---

# Sử dụng cho dữ liệu NA_Fish_Dataset

In [21]:
data_test = []
# path = "D:\\study\\machine_learning\\classification_fish\\data\\NA_Fish_Dataset"
path1 = 'D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_raw\\NA_Fish_Dataset'
folders = os.listdir(path1)
for folder in folders:
    folder_path = os.path.join(path1, folder)
    num_images = len(os.listdir(folder_path))  # Đếm số lượng ảnh trong thư mục
    for i in range(1, num_images + 1):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        data_test.append([img, folder])


In [22]:
images = []
names = []

for image, name in data_test:
    images.append(image)
    names.append(name)

In [23]:
# chuấn hóa label thành dạng số
le = LabelEncoder()
names = le.fit_transform(names)

# in ra label và tên tương ứng
for i in range(9):
    print(i, le.classes_[i])

0 Black Sea Sprat
1 Gilt Head Bream
2 Hourse Mackerel
3 Red Mullet
4 Red Sea Bream
5 Sea Bass
6 Shrimp
7 Striped Red Mullet
8 Trout


In [24]:
images = np.array(images)
images.shape

(430, 128, 128, 3)

In [25]:
model = load_model('D:\\study-hk2\\ML\\ML\\classification_fish\\model\\segmeantation\\unet_1.hdf5', 
                   custom_objects={'my_iou_metric': my_iou_metric})

In [26]:
preds = model.predict(images, verbose=1)

14/14 [==============================] - 5s 358ms/step


In [27]:
preds = (preds > 0.5).astype(np.uint8)

In [28]:
import os
path_gt = "D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_fix\\NA_Fish_Dataset_GT"
if not os.path.exists(path_gt):
    os.makedirs(path_gt)

In [29]:
# Xác định các thư mục 
folder_names = ['Black Sea Sprat', 'Gilt Head Bream', 'Hourse Mackerel', 'Red Mullet',
               'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']

# Tạo các thư mục nếu nó chưa tồn tại
for folder_name in folder_names:
    folder_path = os.path.join(path_gt, folder_name)
    os.makedirs(folder_path, exist_ok=True)

In [30]:
preds = np.squeeze(preds)

In [32]:
# Lặp qua các hình ảnh
for i in range(len(preds)):
    # Lấy tên thư mục và chỉ mục ảnh
    folder_index = i // 50
    folder_name = folder_names[folder_index]
    image_index = i % 50

  # Định dạng chỉ mục ảnh dưới dạng chuỗi 5 chữ số
    image_index_str = str(image_index + 1).zfill(5)

    # Tạo tên ảnh
    filename = f'{image_index_str}.png'

    # Chuyển đổi về dạng nhị phân
    img_binary = (preds[i] > 0.5).astype(np.uint8) * 255

    # Tạo đối tượng từ mảng nhị phân
    pred_image = Image.fromarray(img_binary, mode='L')

    # Lưu các ảnh
    folder_path = os.path.join(path_gt, folder_name)
    output_path = os.path.join(folder_path, filename)
    pred_image.save(output_path)

In [33]:
path_img = "D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_raw\\NA_Fish_Dataset"
path_gt = "D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_fix\\NA_Fish_Dataset_GT"
output_folder = "D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_fix\\NA_Fish_Dataset_Segment"

In [34]:
# Define the folder names
folder_names = ['Black Sea Sprat', 'Gilt Head Bream', 'Hourse Mackerel', 'Red Mullet',
               'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']

# Create the folders if they don't exist
for folder_name in folder_names:
    folder_path = os.path.join(output_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)

In [39]:
folders = os.listdir(path_img)
for folder in folders:
    folder_path = os.path.join(path_img, folder)
    folder_gt = os.path.join(path_gt, folder)
    print(folder_path)
    print(folder_gt)
    num_images = len(os.listdir(folder_path)) 
    for i in range(1, num_images+1):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")
        mask_path = os.path.join(folder_gt, f"{str(i).zfill(5)}.png")
        # đọc ảnh
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        mask = cv2.imread(mask_path)
        mask = cv2.resize(mask, (128, 128))

        # Chuyển ảnh mask sang ảnh binary
        gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)

        obj = np.zeros_like(img)

        # Tìm contours trên ảnh binary
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # Lặp qua từng contour
        for contour in contours:
            # Tìm bounding box của contour
            x, y, w, h = cv2.boundingRect(contour)

            # Kiểm tra kích thước bounding box
            if w > 5 and h > 5:
                # Kiểm tra màu của pixel tương ứng trên ảnh ground truth
                if mask[y:y + h, x:x + w].max() == 255:
                    # Lấy phần ảnh tương ứng từ ảnh gốc
                    obj[y:y + h, x:x + w] = cv2.bitwise_and(img[y:y + h, x:x + w], mask[y:y + h, x:x + w])
        
        # Lưu ảnh obj vào thư mục D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset_Segment\{folder}
        cv2.imwrite(f"D:\\study-hk2\\ML\\ML\\classification_fish\\data\\data_fix\\NA_Fish_Dataset_Segment\\{folder}\\{str(i).zfill(5)}.png", obj)

D:\study-hk2\ML\ML\classification_fish\data\data_raw\NA_Fish_Dataset\Black Sea Sprat
D:\study-hk2\ML\ML\classification_fish\data\data_fix\NA_Fish_Dataset_GT\Black Sea Sprat
D:\study-hk2\ML\ML\classification_fish\data\data_raw\NA_Fish_Dataset\Gilt Head Bream
D:\study-hk2\ML\ML\classification_fish\data\data_fix\NA_Fish_Dataset_GT\Gilt Head Bream
D:\study-hk2\ML\ML\classification_fish\data\data_raw\NA_Fish_Dataset\Hourse Mackerel
D:\study-hk2\ML\ML\classification_fish\data\data_fix\NA_Fish_Dataset_GT\Hourse Mackerel
D:\study-hk2\ML\ML\classification_fish\data\data_raw\NA_Fish_Dataset\Red Mullet
D:\study-hk2\ML\ML\classification_fish\data\data_fix\NA_Fish_Dataset_GT\Red Mullet
D:\study-hk2\ML\ML\classification_fish\data\data_raw\NA_Fish_Dataset\Red Sea Bream
D:\study-hk2\ML\ML\classification_fish\data\data_fix\NA_Fish_Dataset_GT\Red Sea Bream
D:\study-hk2\ML\ML\classification_fish\data\data_raw\NA_Fish_Dataset\Sea Bass
D:\study-hk2\ML\ML\classification_fish\data\data_fix\NA_Fish_Dataset_GT